In [1]:
import os

# Grabbing data with datalad
[Datalad](https://www.datalad.org/) is a git and git-annex based method of version control for data management, data sharing and reproducible science. For this lesson, it offers us a convenient way of grabbing the data that we need for the lesson. These comamands import datald so that we can use it to clone the repository into our virtual machine.

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
import datalad.api as dl

In [5]:
dl_source='/Users/davecash/Data/IDEAS/sample'

In [6]:
sample=dl.clone(dl_source,path='/tmp/sample',description='Cloned sample dataset for import')

In [7]:
bl_img=sample.get('./baseline_t1.nii.gz')

In [8]:
fu_img=sample.get('./followup_t1.nii.gz')

In [9]:
print(bl_img)

[{'action': 'get', 'path': '/tmp/sample/baseline_t1.nii.gz', 'type': 'file', 'refds': '/tmp/sample', 'status': 'notneeded', 'message': 'already present'}]


# FSL eyes setup

This *magic* command is needed to tie the wxWidgets backend used by FSLeyes to the notebook. Note that there are some steps to make this work locally, please consult the FSLeyes documentation about [linking to a Jupyter notebook](https://open.win.ox.ac.uk/pages/fsl/fsleyes/fsleyes/userdoc/fsleyes_notebook.html) and [troubleshooting for Mac](https://open.win.ox.ac.uk/pages/fsl/fsleyes/fsleyes/userdoc/troubleshooting.html#macos-i-can-t-start-fsleyes-from-ipython-jupyter-notebook) for more information.

In [10]:
%gui wx

Next, we initialise FSLeyes and create the frame, or window that we will be starting the viewer in.

In [11]:
import fsleyes
overlayList, displayCtx, frame = fsleyes.embed()
frame.Show()

15:00:29: Debug: Adding duplicate image handler for 'Windows bitmap file'
15:00:29: Debug: Adding duplicate animation handler for '1' type
15:00:29: Debug: Adding duplicate animation handler for '2' type
15:00:31: Debug: Adding duplicate image handler for 'Windows bitmap file'
15:00:31: Debug: Adding duplicate animation handler for '1' type
15:00:31: Debug: Adding duplicate animation handler for '2' type
/Users/davecash/anaconda3/envs/demon-imaging-harmonisation/lib/python3.7/site-packages/fsleyes/colourmaps.py:572: UserWarning: Trying to register the cmap 'pink' which already exists.
  mplcm.register_cmap(key, cmap, override_builtin=True)
/Users/davecash/anaconda3/envs/demon-imaging-harmonisation/lib/python3.7/site-packages/fsleyes/colourmaps.py:572: UserWarning: Trying to register the cmap 'hot' which already exists.
  mplcm.register_cmap(key, cmap, override_builtin=True)
/Users/davecash/anaconda3/envs/demon-imaging-harmonisation/lib/python3.7/site-packages/fsleyes/colourmaps.py:572:

True

Then we grab the image data that is needed for viewing. We first create an O

In [12]:
from fsleyes.views.orthopanel import OrthoPanel
from fsl.data.image import Image
ortho = frame.addViewPanel(OrthoPanel)

In [18]:
overlayList.append(Image(bl_img[0]['path']))
overlayList.append(Image(fu_img[0]['path']))

In [19]:
frame.Close()
fsleyes.shutdown()